In [1]:
import re
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Вспомогательные функции
def check_for_none(x):
    if x is None:
        return ''
    else:
        return x.text

# Parsing

## Avito

In [3]:
main_url = 'https://www.avito.ru'
add_url = '/moskva_i_mo/avtomobili?p=1'

In [11]:
pred_ap = []
full_ap = []

# Короткий текст объявления в шаблоне поиска
url = main_url + add_url
r = requests.get(url)
soup = BeautifulSoup(r.text)
ap = soup.findAll('div', {'class': 'snippet-horizontal'})
pred_ap.append({
    'title': check_for_none(ap[0].find('div', {'class': 'snippet-title-row'})),
    'price_comment': check_for_none(ap[0].find('span', {'class': 'snippet-tag'})),
    'price': check_for_none(ap[0].find('span', {'class': 'snippet-price'})),
    'specific-params':  check_for_none(ap[0].find('div', {'class': 'specific-params_block'})),
    'address': check_for_none(ap[0].find('div', {'class': 'item-address'})), 
})
########################################################################
# Полный текст внутри объявления
full_app = []
full_app_dict = {}

url = main_url + ap[0].find('a', {'class': 'snippet-link'}, href=True)['href']
r = requests.get(url)
soup = BeautifulSoup(r.text)


fap = soup.find('div', {'class': 'item-params'})

li = fap.findAll('li', {'class':'item-params-list-item'})
for l in li:
    full_app_dict[l.text.split(':')[0].strip()] = l.text.split(':')[1].strip()
full_app_dict['address'] = soup.find('div', {'class':'item-address'}).text
full_app_dict['full_text_app'] = soup.find('div', class_=re.compile('item-description')).text

# Дополенная информация об объекте
adv_info = {}

fap_adv = soup.find('ul', class_=re.compile('advanced-params'))
if fap_adv is not None:
    li = fap_adv.findAll('li', {'class': 'advanced-params-param'})
    for l in li:
        param_list = []
        title = l.find('div', {'class': 'advanced-params-param-title'}).text
        item_list = l.find('ul', {'class': 'advanced-params-param-list'})  
        for il in item_list.findAll('li', {'class': 'advanced-params-param-item'}):
                param_list.append(il.text)    
        adv_info[title] = param_list

#Дополнительная информация по цене в объявлении  
add_price = []

js = soup.find('div', {'class':'js-item-imv-ssr'})
if js is not None:    
    add_info = json.loads(js['data-props'])
    add_price.append({
        'title': add_info['title'],
        'subtitle': add_info['subtitle'],
        'priceType': add_info['priceType'],
        'marketPrice': add_info['marketPrice'],
        'isVisibleForUser': add_info['isVisibleForUser'],
        'price': add_info['price']
    })
else:
    add_price.append({
        'title': '',
        'subtitle': '',
        'priceType': '',
        'marketPrice': '',
        'isVisibleForUser': '',
        'price': ''
    })

#Дополнительная проверка по VIN
#vin = soup.find('div', {'class':'list-list-2pUcD'})

https://www.avito.ru/himki/avtomobili/kia_sorento_prime_2020_1914121802
None


In [12]:
add_price

[{'title': '',
  'subtitle': '',
  'priceType': '',
  'marketPrice': '',
  'isVisibleForUser': '',
  'price': ''}]

In [13]:
pred_ap

[{'title': 'KIA Sorento Prime, 2020Новый',
  'price_comment': 'Новый',
  'price': '\n 2 064 900  ₽\n  ',
  'specific-params': '\n  2.4 AT (188 л.с.), внедорожник, полный, бензин\n ',
  'address': '\nХимки\n'}]

In [14]:
full_app

[]

In [15]:
adv_info

{'Усилитель руля': ['Электрогидроусилитель'],
 'Управление климатом': ['Климат однозонный', 'Атермальное остекление'],
 'Салон': ['Кожа', 'Кожаный руль'],
 'Обогрев': ['Передних сидений', 'Задних сидений', 'Зеркал', 'Руля'],
 'Электростеклоподъемники': ['Передние и задние'],
 'Электропривод': ['Передних сидений', 'Зеркал'],
 'Помощь при вождении': ['Датчик дождя',
  'Датчик света',
  'Камера заднего вида',
  'Круиз-контроль',
  'Бортовой компьютер'],
 'Противоугонная система': ['Сигнализация',
  'Центральный замок',
  'Иммобилайзер'],
 'Подушки безопасности': ['Фронтальные', 'Шторки', 'Боковые передние'],
 'Активная безопасность': ['Антиблокировочная система тормозов (ABS)',
  'Антипробуксовочная система (ASR)',
  'Система курсовой устойчивости (ESP/ESC/DSC)',
  'Система экстренного торможения (EBA/BAS/BA)'],
 'Мультимедиа и навигация': ['CD/DVD/Blu-ray',
  'MP3',
  'Радио',
  'Управление на руле'],
 'Фары': ['Светодиодные', 'Противотуманные', 'Омыватели фар'],
 'Шины и диски': ['17"']

In [16]:
full_app_dict

{'Марка': 'KIA',
 'Модель': 'Sorento Prime',
 'Поколение': 'III рестайлинг (2017—н. в.)',
 'Модификация': '2.4 4WD AT (188 л.с.)',
 'Год выпуска': '2020',
 'VIN или номер кузова': 'XWEP*************',
 'Тип кузова': 'внедорожник',
 'Количество дверей': '5',
 'Тип двигателя': 'бензин',
 'Коробка передач': 'автомат',
 'Привод': 'полный',
 'Руль': 'левый',
 'Цвет': 'чёрный',
 'Комплектация': 'Luxe',
 'Место осмотра': 'Московская область, Химки, Ленинградское шоссе, вл7',
 'address': '\n\n Московская область, Химки, Ленинградское шоссе, вл7\n \n',
 'full_text_app': ' \nУспейте купить Новый KIA Sorento Prime со скидкой до 330 000 рублей! Уникальное предложение при приобретении автомобиля в Киа Центр Шереметьево: 1. Скидка от дилерского центра при приобретении за наличный расчет - до 40 000 рублей. 2. Скидка по программе Трейд-ин (обмен Вашего автомобиля на KIA Sorento Prime) - до 190 000 рублей. 3. Скидка при приобретении автомобиля в кредит - до 100 000 рублей. 4. Дополнительная выгода при